# import libraries

In [91]:
import pandas as pd

# file operations

In [92]:
def read_csv_file(path):
    df = pd.read_csv(path)
    return df

# Read Data

In [93]:
df = read_csv_file('result.csv')
print(df.info())
# df['result'] = 0
# for i in range(5):
#     df['result'] += df['prompt{}'.format(i)]
# df['result'] /= 5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          500 non-null    object 
 1   label         500 non-null    int64  
 2   result        500 non-null    float64
 3   prompt0_meta  500 non-null    object 
 4   prompt0       500 non-null    int64  
 5   prompt1_meta  500 non-null    object 
 6   prompt1       500 non-null    int64  
 7   prompt2_meta  500 non-null    object 
 8   prompt2       500 non-null    int64  
 9   prompt3_meta  500 non-null    object 
 10  prompt3       500 non-null    int64  
 11  prompt4_meta  500 non-null    object 
 12  prompt4       500 non-null    int64  
 13  prompt5_meta  500 non-null    object 
 14  prompt5       500 non-null    int64  
 15  prompt6_meta  500 non-null    object 
 16  prompt6       500 non-null    int64  
 17  prompt7_meta  500 non-null    object 
 18  prompt7       500 non-null    

# Analyze Data

In [101]:
def analyze(df, threshold):
    df["hypothesis"] = df["result"].apply(lambda x: 1 if x >= threshold else 0)

    count_true_positive = len(df[(df['hypothesis'] == 1) & (df['label'] == 0)])
    count_true_negative = len(df[(df['hypothesis'] == 0) & (df['label'] != 0)])
    count_false_positive = len(df[(df['hypothesis'] == 1) & (df['label'] != 0)])
    count_false_negative = len(df[(df['hypothesis'] == 0) & (df['label'] == 0)])
    # print("true positive: {count}".format(count=count_true_positive))
    # print("true negative: {count}".format(count=count_true_negative))
    # print("false positive: {count}".format(count=count_false_positive))
    # print("false negative: {count}".format(count=count_false_negative))

    print("precision = {}".format(round(count_true_positive / (count_true_positive + count_false_positive), 4)))
    print("recall = {}".format(round(count_true_positive / (count_true_positive + count_false_negative), 4)))
    print("accuracy = {}".format(round((count_true_positive + count_true_negative) / (count_true_positive + count_true_negative + count_false_positive + count_false_negative), 4)))

    real_positive_accuarcy = []
    estimate_positive_accuracy = []
    real_accuracy = []
    estimate_accuracy = []
    for prompt_id in range(10):
        print(prompt_id)
        real_count_true_positive = len(df[(df['prompt{}'.format(prompt_id)] == 1) & (df['label'] == 0)])
        real_count_true_negative = len(df[(df['prompt{}'.format(prompt_id)] == 0) & (df['label'] != 0)])
        real_count_false_positive = len(df[(df['prompt{}'.format(prompt_id)] == 1) & (df['label'] != 0)])
        real_count_false_negative = len(df[(df['prompt{}'.format(prompt_id)] == 0) & (df['label'] == 0)])
        # print("prompt {prompt_id} real true positive: {count}".format(prompt_id=prompt_id, count=real_count_true_positive))
        # print("prompt {prompt_id} real true negative: {count}".format(prompt_id=prompt_id, count=real_count_true_negative))
        # print("prompt {prompt_id} real false positive: {count}".format(prompt_id=prompt_id, count=real_count_false_positive))
        # print("prompt {prompt_id} real false negative: {count}".format(prompt_id=prompt_id, count=real_count_false_negative))
        print("precision = {}".format(round(real_count_true_positive / (real_count_true_positive + real_count_false_positive), 4)))
        print("recall = {}".format(round(real_count_true_positive / (real_count_true_positive + real_count_false_negative), 4)))
        print("accuracy = {}".format(round((real_count_true_positive + real_count_true_negative) / (real_count_true_positive + real_count_true_negative + real_count_false_positive + real_count_false_negative), 4)))
        # print("prompt {prompt_id}".format(prompt_id=prompt_id), round(real_count_true_positive / (real_count_true_positive + real_count_false_positive), 4))
        real_positive_accuarcy.append(real_count_true_positive / (real_count_true_positive + real_count_false_positive))
        real_accuracy.append((real_count_true_positive + real_count_true_negative) / (real_count_true_positive + real_count_true_negative + real_count_false_positive + real_count_false_negative))
        estimate_count_true_positive = len(df[(df['prompt{}'.format(prompt_id)] == 1) & (df['hypothesis'] == 1)])
        estimate_count_true_negative = len(df[(df['prompt{}'.format(prompt_id)] == 0) & (df['hypothesis'] == 0)])
        estimate_count_false_positive = len(df[(df['prompt{}'.format(prompt_id)] == 1) & (df['hypothesis'] == 0)])
        estimate_count_false_negative = len(df[(df['prompt{}'.format(prompt_id)] == 0) & (df['hypothesis'] == 1)])
        # print("prompt {prompt_id} estimate true positive: {count}".format(prompt_id=prompt_id, count=estimate_count_true_positive))
        # print("prompt {prompt_id} estimate true negative: {count}".format(prompt_id=prompt_id, count=estimate_count_true_negative))
        # print("prompt {prompt_id} estimate false positive: {count}".format(prompt_id=prompt_id, count=estimate_count_false_positive))
        # print("prompt {prompt_id} estimate false negative: {count}".format(prompt_id=prompt_id, count=estimate_count_false_negative))

        # print("prompt {prompt_id}".format(prompt_id=prompt_id), round(estimate_count_true_positive / (estimate_count_true_positive + estimate_count_false_positive), 4))
        estimate_positive_accuracy.append(estimate_count_true_positive / (estimate_count_true_positive + estimate_count_false_positive))
        estimate_accuracy.append((estimate_count_true_positive + estimate_count_true_negative) / (estimate_count_true_positive + estimate_count_true_negative + estimate_count_false_positive + estimate_count_false_negative))
    print("max real positive accuracy: {}".format(max(real_positive_accuarcy)))
    print("min real positive accuracy: {}".format(min(real_positive_accuarcy)))
    print("max real accuracy: {}".format(max(real_accuracy)))
    print("min real accuracy: {}".format(min(real_accuracy)))

    index_max_estimate_positive_accuracy = estimate_positive_accuracy.index(max(estimate_positive_accuracy))
    print("max estimate positive accuracy: {}".format(real_positive_accuarcy[index_max_estimate_positive_accuracy]))
    index_max_estimate_accuracy = estimate_accuracy.index(max(estimate_accuracy))
    print("max estimate accuracy: {}".format(real_accuracy[index_max_estimate_accuracy]))
    print("real positive accuracy: {}".format(real_positive_accuarcy))
    print("estimate positive accuracy: {}".format(estimate_positive_accuracy))
    print("real accuracy: {}".format(real_accuracy))
    print("estimate accuracy: {}".format(estimate_accuracy))
    
test_df = df
# test_df = df.sample(n=300)
for threshold in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print("threshold: {threshold}".format(threshold=threshold))
    analyze(test_df, threshold)
    print("--------------------------------------------------")


threshold: 0.5
precision = 0.6384
recall = 0.7588
accuracy = 0.694
0
precision = 0.7277
recall = 0.6798
accuracy = 0.738
1
precision = 0.6694
recall = 0.7193
accuracy = 0.71
2
precision = 0.7198
recall = 0.5746
accuracy = 0.704
3
precision = 0.6175
recall = 0.7719
accuracy = 0.678
4
precision = 0.5452
recall = 0.7412
accuracy = 0.6
5
precision = 0.7064
recall = 0.7281
accuracy = 0.738
6
precision = 0.6258
recall = 0.8289
accuracy = 0.696
7
precision = 0.6148
recall = 0.7281
accuracy = 0.668
8
precision = 0.5116
recall = 0.7719
accuracy = 0.56
9
precision = 0.5424
recall = 0.8421
accuracy = 0.604
max real positive accuracy: 0.7276995305164319
min real positive accuracy: 0.5116279069767442
max real accuracy: 0.738
min real accuracy: 0.56
max estimate positive accuracy: 0.7197802197802198
max estimate accuracy: 0.678
real positive accuracy: [0.7276995305164319, 0.6693877551020408, 0.7197802197802198, 0.6175438596491228, 0.5451612903225806, 0.7063829787234043, 0.6258278145695364, 0.6148148

In [95]:
import numpy as np
m = 500
n = 10
# test_df = df
m = test_df.shape[0]
X = np.array([[1 if row["prompt{}".format(i)] == 1 else -1 for i in range(10)] for index, row in test_df.iterrows()])
print(X.shape)

(500, 10)


In [96]:
# SVD to find first singular vector
from sklearn.utils.extmath import randomized_svd
# U, s, Vt = np.linalg.svd(X, full_matrices=False)
# print(Vt)
U, s, Vt = randomized_svd(X, n_components=1)
print(Vt)
Vt = Vt * s[0] * np.sign(Vt[0][0])
w_hat = Vt / np.sqrt(m)
w_tilde = w_hat
w_tilde[w_tilde > 1] = 1
w_tilde[w_tilde < -1] = -1

q_hat = np.sign(np.matmul(X, w_tilde[0]))

print(q_hat.shape)

test_df["estimate_result"] = q_hat

count_true_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] == 0)])
count_true_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] != 0)])
count_false_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] != 0)])
count_false_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] == 0)])
print("true positive: {count}".format(count=count_true_positive))
print("true negative: {count}".format(count=count_true_negative))
print("false positive: {count}".format(count=count_false_positive))
print("false negative: {count}".format(count=count_false_negative))

print("precision: {precision}".format(precision=count_true_positive / (count_true_positive + count_false_positive)))
print("recall: {recall}".format(recall=count_true_positive / (count_true_positive + count_false_negative)))
print("accuracy: {accuracy}".format(accuracy=(count_true_positive + count_true_negative) / len(test_df)))

print(w_hat)


[[0.33096849 0.36536063 0.30150423 0.36069314 0.24907924 0.34832095
  0.33338407 0.35669106 0.16787392 0.29333269]]
(500,)
true positive: 171
true negative: 179
false positive: 93
false negative: 57
precision: 0.6477272727272727
recall: 0.75
accuracy: 0.7
[[0.80720182 0.89108109 0.73534118 0.87969751 0.60748143 0.84952288
  0.8130932  0.86993679 0.40942911 0.71541156]]


In [97]:
test_df = df.sample(n=100)
for threshold in [0.7]:
    print("threshold: {threshold}".format(threshold=threshold))
    analyze(test_df, threshold)
    print("--------------------------------------------------")
n = 10
# test_df = df
m = test_df.shape[0]
X = np.array([[1 if row["prompt{}".format(i)] == 1 else -1 for i in range(10)] for index, row in test_df.iterrows()])
print(X.shape)
# SVD to find first singular vector
# U, s, V = np.linalg.svd(X, full_matrices=False)
U, s, Vt = randomized_svd(X, n_components=1)
Vt = Vt * s[0] * np.sign(Vt[0][0])
w_hat = Vt / np.sqrt(m)
w_tilde = w_hat
w_tilde[w_tilde > 1] = 1
w_tilde[w_tilde < -1] = -1

q_hat = np.sign(np.matmul(X, w_tilde[0]))

print(q_hat.shape)

test_df["estimate_result"] = q_hat

count_true_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] == 0)])
count_true_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] != 0)])
count_false_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] != 0)])
count_false_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] == 0)])
print("true positive: {count}".format(count=count_true_positive))
print("true negative: {count}".format(count=count_true_negative))
print("false positive: {count}".format(count=count_false_positive))
print("false negative: {count}".format(count=count_false_negative))

print("precision: {precision}".format(precision=count_true_positive / (count_true_positive + count_false_positive)))
print("recall: {recall}".format(recall=count_true_positive / (count_true_positive + count_false_negative)))
print("accuracy: {accuracy}".format(accuracy=(count_true_positive + count_true_negative) / len(test_df)))

print(w_hat)



threshold: 0.7
true positive / true positive + false positive = 0.6757
accuracy = 0.76
max real positive accuracy: 0.6956521739130435
min real positive accuracy: 0.41935483870967744
max real accuracy: 0.77
min real accuracy: 0.53
max estimate positive accuracy: 0.6956521739130435
max estimate accuracy: 0.77
real positive accuracy: [0.6944444444444444, 0.5952380952380952, 0.6956521739130435, 0.5777777777777777, 0.4807692307692308, 0.6486486486486487, 0.6222222222222222, 0.5909090909090909, 0.41935483870967744, 0.4393939393939394]
estimate positive accuracy: [0.9444444444444444, 0.8809523809523809, 1.0, 0.8, 0.6346153846153846, 0.918918918918919, 0.8, 0.8409090909090909, 0.5, 0.5606060606060606]
real accuracy: [0.77, 0.71, 0.72, 0.7, 0.61, 0.74, 0.74, 0.71, 0.53, 0.55]
estimate accuracy: [0.95, 0.95, 0.86, 0.9, 0.77, 0.94, 0.9, 0.93, 0.63, 0.71]
--------------------------------------------------
(100, 10)
(100,)
true positive: 25
true negative: 49
false positive: 14
false negative: 12
pr